In [50]:
import pandas as pd

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

ratings = pd.read_csv("u.data", sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
users_movies = pd.read_csv("u.item", sep='|', header=None, encoding='latin-1')
users_movies = users_movies[[0, 1]]
users_movies.columns = ['movie_id', 'title']


In [51]:
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

movies = movies.merge(credits, left_on="id", right_on="movie_id")

movies = movies[["title_x", "overview", "genres", "keywords", "cast", "crew"]]
movies.rename(columns={"title_x": "title"}, inplace=True)

def convert(obj):
    try:
        obj = ast.literal_eval(obj)
    except:
        return []
    return [i["name"] for i in obj]

def get_director(obj):
    try:
        obj = ast.literal_eval(obj)
    except:
        return []
    for i in obj:
        if i['job'] == 'Director':
            return [i['name']]
    return []

movies["genres"] = movies["genres"].apply(convert)
movies["keywords"] = movies["keywords"].apply(convert)
movies["cast"] = movies["cast"].apply(lambda x: convert(x)[:3])
movies["director"] = movies["crew"].apply(get_director)

movies["overview"] = movies["overview"].fillna("")

for col in ["genres","keywords","cast","director"]:
    movies[col] = movies[col].apply(lambda x: [i.replace(" ", "").lower() for i in x])

movies["tags"] = (
    movies["overview"].str.lower() + " " +
    movies["genres"].apply(" ".join) + " " +
    movies["keywords"].apply(" ".join) + " " +
    movies["cast"].apply(" ".join) + " " +
    movies["director"].apply(" ".join)
)

new_df = movies[["title", "tags"]]
cv = CountVectorizer(max_features=5000, stop_words="english")
vectors = cv.fit_transform(new_df["tags"]).toarray()
similarity = cosine_similarity(vectors)


In [52]:
def content_recommend(movie):
    movie = movie.lower()
    index = new_df[new_df.title.str.lower() == movie].index[0]
    distances = sorted(enumerate(similarity[index]), key=lambda x: x[1], reverse=True)[:6]
    for i in distances[1:]:
        print(new_df.iloc[i[0]].title)

content_recommend("Avatar")


Titan A.E.
Small Soldiers
Independence Day
Aliens vs Predator: Requiem
Ender's Game


Content based Successfully done

In [53]:
from sklearn.neighbors import NearestNeighbors

user_movie_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)
model_cf = NearestNeighbors(metric='cosine', algorithm='brute')
model_cf.fit(user_movie_matrix.values)


,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [71]:
def collab_recommend(user_id, n=5):
    distances, indices = model_cf.kneighbors(
        user_movie_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=n)
    similar_users = indices.flatten()[1:]
    
    rec = ratings[ratings.user_id.isin(similar_users)] \
        .groupby("movie_id")["rating"].mean() \
        .sort_values(ascending=False)[:10]
    
    return users_movies[users_movies.movie_id.isin(rec.index)]["title"].tolist()
collab_recommend(50)

['Die Hard 2 (1990)',
 'Star Trek VI: The Undiscovered Country (1991)',
 'Star Trek: The Wrath of Khan (1982)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek IV: The Voyage Home (1986)',
 'Batman Returns (1992)',
 'Stand by Me (1986)',
 'M (1931)',
 'Pump Up the Volume (1990)',
 'Aparajito (1956)']

In [56]:
def hybrid_recommend(movie_name, user_id):
    print("\nHYBRID RECOMMENDATIONS:")
    print("-"*40)

    print("\n🎞 Content-based Similar Movies:")
    content_recommend(movie_name)

    print("\n👥 Collaborative Recommendations:")
    print(collab_recommend(user_id))
hybrid_recommend("Avatar", 100)



HYBRID RECOMMENDATIONS:
----------------------------------------

🎞 Content-based Similar Movies:
Titan A.E.
Small Soldiers
Independence Day
Aliens vs Predator: Requiem
Ender's Game

👥 Collaborative Recommendations:
['Twelve Monkeys (1995)', 'Richard III (1995)', 'Hoodlum (1997)', "Devil's Advocate, The (1997)", 'Alien: Resurrection (1997)', 'Jackie Brown (1997)', 'To Kill a Mockingbird (1962)', 'Day the Earth Stood Still, The (1951)', 'Ghost and Mrs. Muir, The (1947)', 'Live Nude Girls (1995)']


In [57]:
from sklearn.decomposition import NMF
import numpy as np

# create user-movie matrix (if not already created)
user_movie_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# convert to numpy array (required for NMF)
matrix = user_movie_matrix.values

# Define NMF model
nmf_model = NMF(n_components=20, init='random', random_state=42)

# Train NMF model
user_factors = nmf_model.fit_transform(matrix)   # shape → (users x 20)
movie_factors = nmf_model.components_           # shape → (20 x movies)

print("NMF model trained successfully!")


NMF model trained successfully!


D:\anaconda\Lib\site-packages\sklearn\decomposition\_nmf.py:1728: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [70]:
def nmf_recommend(user_id, top_n=5):
    # NMF latent factors dot product gives predictions
    user_vector = user_factors[user_id - 1]  # row index starts at 0
    scores = np.dot(user_vector, movie_factors)

    # sort movies by predicted score
    movie_indices = scores.argsort()[::-1]   # descending order

    # get top movie_ids
    recommended_movie_ids = movie_indices[:top_n]

    # map movie IDs to movie titles
    recommended_titles = users_movies[users_movies.movie_id.isin(recommended_movie_ids)]
    
    return recommended_titles["title"].tolist()
nmf_recommend(50, top_n=5)


["Devil's Own, The (1997)",
 'unknown',
 'Sense and Sensibility (1995)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Streetcar Named Desire, A (1951)']

In [63]:
import pandas as pd

# Load MovieLens data
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Load movie titles
movies_info = pd.read_csv('u.item', sep='|', header=None, encoding='latin-1')
movies_info = movies_info[[0, 1]]
movies_info.columns = ['movie_id', 'title']

# Create user-movie matrix (rows = users, columns = movies)
user_movie_matrix = ratings.pivot_table(
    index='user_id', 
    columns='movie_id', 
    values='rating'
).fillna(0)

user_movie_matrix.head()


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
import numpy as np

R = user_movie_matrix.values.astype('float32')
R_scaled = R / 5.0    # since ratings are 1–5


In [67]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_users, num_movies = R_scaled.shape
encoding_dim = 64   # latent dimensionality

input_layer = layers.Input(shape=(num_movies,))

# Encoder
encoded = layers.Dense(256, activation='relu')(input_layer)
encoded = layers.Dense(128, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

# Decoder
decoded = layers.Dense(128, activation='relu')(encoded)
decoded = layers.Dense(256, activation='relu')(decoded)
decoded = layers.Dense(num_movies, activation='sigmoid')(decoded)  # outputs in [0,1]

autoencoder = models.Model(inputs=input_layer, outputs=decoded)

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1682)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         430,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1682)                │         432,274 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 945,618 (3.61 MB)

 Trainable params: 945,618 (3.61 MB)

 Non-trainable params: 0 (0.00 B)

In [68]:
history = autoencoder.fit(
    R_scaled,            # input = output
    R_scaled,
    epochs=25,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.1695 - val_loss: 0.0340
Epoch 2/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0329 - val_loss: 0.0346
Epoch 3/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0329 - val_loss: 0.0345
Epoch 4/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0328 - val_loss: 0.0347
Epoch 5/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0335 - val_loss: 0.0346
Epoch 6/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0349 - val_loss: 0.0346
Epoch 7/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0329 - val_loss: 0.0344
Epoch 8/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0331 - val_loss: 0.0343
Epoch 9/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0336 - val_loss: 0.0342
Epoch 10/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0340 - val_loss: 0.0339
Epoch 11/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0343 - val_loss: 0.0337
Epoch 12/25
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0329 - val_l

In [69]:
def ae_recommend(user_id, top_n=10):
    # user_id starts from 1 but numpy index from 0
    user_index = user_id - 1
    
    # Get user's original rating vector
    user_ratings = R_scaled[user_index].reshape(1, -1)
    
    # Get reconstructed (predicted) ratings
    reconstructed = autoencoder.predict(user_ratings, verbose=0)[0]
    
    # Don't recommend already-rated movies (non-zero original ratings)
    already_rated = R[user_index] > 0
    reconstructed[already_rated] = 0.0
    
    # Get indices of top N predicted movies
    top_indices = reconstructed.argsort()[::-1][:top_n]
    
    # Movie IDs are the columns of user_movie_matrix
    movie_ids = user_movie_matrix.columns.values[top_indices]
    
    # Map to titles
    recommended = movies_info[movies_info.movie_id.isin(movie_ids)]
    
    return recommended.title.values
ae_recommend(50, top_n=5)


array(['Star Wars (1977)', 'Godfather, The (1972)', 'Contact (1997)',
       'Full Monty, The (1997)', 'L.A. Confidential (1997)'], dtype=object)

Model Training for exporting

In [72]:
import pandas as pd
import numpy as np
import ast
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras import layers, models


In [73]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
ratings = pd.read_csv("u.data", sep="\t", names=["user_id","movie_id","rating","timestamp"])
movie_titles = pd.read_csv("u.item", sep="|", header=None, encoding="latin-1", usecols=[0,1])
movie_titles.columns = ["movie_id","title"]


In [74]:
movies = movies.merge(credits, left_on="id", right_on="movie_id")

movies = movies[["title_x","overview","genres","keywords","cast","crew"]]
movies.rename(columns={"title_x":"title"}, inplace=True)

def clean_list(x):
    try:
        x = ast.literal_eval(x)
        return [i["name"].replace(" ","").lower() for i in x][:3]
    except:
        return []

def director(x):
    try:
        x = ast.literal_eval(x)
        for i in x:
            if i["job"] == "Director":
                return [i["name"].replace(" ","").lower()]
        return []
    except:
        return []

movies["genres"] = movies["genres"].apply(clean_list)
movies["keywords"] = movies["keywords"].apply(clean_list)
movies["cast"] = movies["cast"].apply(clean_list)
movies["director"] = movies["crew"].apply(director)
movies["overview"] = movies["overview"].fillna("").str.lower()

movies["tags"] = (
    movies["overview"] + " " +
    movies["genres"].apply(" ".join) + " " +
    movies["keywords"].apply(" ".join) + " " +
    movies["cast"].apply(" ".join) + " " +
    movies["director"].apply(" ".join)
)


In [75]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(movies["tags"]).toarray()
similarity = cosine_similarity(vectors)


In [76]:
import pickle

pickle.dump(cv, open("vectorizer.pkl", "wb"))
pickle.dump(similarity, open("content_similarity.pkl", "wb"))
pickle.dump(movies[["title"]], open("movie_list.pkl", "wb"))


Collaborative Filtering

In [78]:
from sklearn.neighbors import NearestNeighbors
import pickle

# Create User-Movie Matrix
user_movie_matrix = ratings.pivot_table(
    index='user_id', columns='movie_id', values='rating'
).fillna(0)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_movie_matrix.values)


,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [79]:
pickle.dump(knn_model, open("knn_model.pkl", "wb"))
pickle.dump(user_movie_matrix, open("user_movie_matrix.pkl", "wb"))
pickle.dump(movie_titles, open("movie_titles.pkl", "wb"))


In [80]:
from sklearn.decomposition import NMF
import numpy as np

# Normalize Ratings (0-1 range for better factorization)
R = user_movie_matrix.values.astype("float32")
R_scaled = R / 5.0

nmf_model = NMF(n_components=20, init="random", random_state=42)
user_factors = nmf_model.fit_transform(R_scaled)
movie_factors = nmf_model.components_


D:\anaconda\Lib\site-packages\sklearn\decomposition\_nmf.py:1728: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [81]:
np.save("nmf_user_factors.npy", user_factors)
np.save("nmf_movie_factors.npy", movie_factors)


In [82]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_users, num_movies = R_scaled.shape
encoding_dim = 64

inp = layers.Input(shape=(num_movies,))
x = layers.Dense(256, activation='relu')(inp)
x = layers.Dense(128, activation='relu')(x)
encoded = layers.Dense(encoding_dim, activation='relu')(x)
x = layers.Dense(128, activation='relu')(encoded)
x = layers.Dense(256, activation='relu')(x)
out = layers.Dense(num_movies, activation='sigmoid')(x)

autoencoder = models.Model(inp, out)
autoencoder.compile(optimizer='adam', loss='mse')


In [83]:
autoencoder.fit(R_scaled, R_scaled, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.1599
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0343
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0322
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0348
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0350
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0337
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0335
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0334
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0349
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0330


In [84]:
autoencoder.save("autoencoder_cf.h5")


In [1]:
import pandas as pd
import mysql.connector
import ast
import os
# Load dataset
df = pd.read_csv("D:/movie-recommender/data/tmdb_5000_movies.csv")

# Connect to MySQL
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Harsha@45",
    database="movie_recommender"
)
cursor = db.cursor()

# Insert movies
for _, row in df.iterrows():
    movie_id = int(row["id"])
    title = row["title"]
    
    # Genres are stored as JSON list, convert to text
    genres = ", ".join([g["name"] for g in ast.literal_eval(row["genres"])])
    
    overview = row["overview"]
    
    # poster path - use TMDB url format
    poster_path = f"https://image.tmdb.org/t/p/w500{row['poster_path']}" if isinstance(row['poster_path'], str) else None
    
    imdb_rating = float(row["vote_average"])
    
    release_year = int(row["release_date"][:4]) if isinstance(row["release_date"], str) and len(row["release_date"]) >= 4 else None

    cursor.execute("""
        INSERT IGNORE INTO movies (id, title, genres, overview, poster_path, imdb_rating, release_year)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
    """, (movie_id, title, genres, overview, poster_path, imdb_rating, release_year))

db.commit()
print("Movies inserted successfully!")


KeyError: 'poster_path'